In [2]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
from scipy.interpolate import griddata

# Load image
img = cv2.imread('/mnt/data/76f5e32f-9442-423d-8546-e3ca5062002b.png')
gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

# Crop to plotting area if needed (adjust manually based on visual check)
cropped = gray[50:420, 60:600]  # [y1:y2, x1:x2]

# Threshold to extract lines (invert for white lines on black)
_, binary = cv2.threshold(cropped, 100, 255, cv2.THRESH_BINARY_INV)

# Find contours
contours, _ = cv2.findContours(binary, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

# Image dimensions for coordinate mapping
h, w = cropped.shape

# Latitude and Altitude ranges
lat_range = (-80, 80)
alt_range = (10, 60)

def pixel_to_data(x, y):
    """Convert pixel (x,y) to (latitude, altitude)"""
    lat = lat_range[0] + (x / w) * (lat_range[1] - lat_range[0])
    alt = alt_range[1] - (y / h) * (alt_range[1] - alt_range[0])
    return lat, alt

# Sample points from contours
data_points = []
contour_levels = [0.2, 0.4, 0.6, 0.8, 1.0]
level_index = 0

for cnt in contours:
    # Approximate the contour to reduce number of points
    approx = cv2.approxPolyDP(cnt, epsilon=2, closed=False)
    for point in approx[::3]:  # downsample points
        x, y = point[0]
        lat, alt = pixel_to_data(x, y)
        value = contour_levels[level_index % len(contour_levels)]
        data_points.append((lat, alt, value))
    level_index += 1

# Convert to array
data_points = np.array(data_points)
lat, alt, values = data_points[:, 0], data_points[:, 1], data_points[:, 2]

# Interpolate onto a grid
lat_grid = np.linspace(-80, 80, 200)
alt_grid = np.linspace(10, 60, 200)
LAT_GRID, ALT_GRID = np.meshgrid(lat_grid, alt_grid)

values_grid = griddata(
    (lat, alt), values, (LAT_GRID, ALT_GRID), method='linear'
)

# Plot the interpolated heatmap
plt.figure(figsize=(8, 6))
cs = plt.contour(LAT_GRID, ALT_GRID, values_grid, levels=contour_levels, colors='black')
plt.clabel(cs, inline=True, fontsize=8)
plt.xlabel("Latitude")
plt.ylabel("Altitude (km)")
plt.title("Reconstructed Contour Plot from Image")
plt.tight_layout()
plt.show()

error: OpenCV(4.11.0) D:\a\opencv-python\opencv-python\opencv\modules\imgproc\src\color.cpp:199: error: (-215:Assertion failed) !_src.empty() in function 'cv::cvtColor'
